In [53]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [54]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
h = hash256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [55]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    h = hash256(item)
    bit = int.from_bytes(h, 'big') % bit_field_size
    bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


### Exercise 1

Calculate the Bloom Filter for 'hello world' and 'goodbye' using the `hash160` hash function over a bit field of 10.

In [56]:
# Exercise 1

from helper import hash160

bit_field_size = 10
bit_field = [0] * bit_field_size
items = (b'hello world', b'goodbye')
# loop through each item
    # hash160 the item
    # interpret hash as a Big-Endian integer and mod by bit_field_size
    # set that bit in bit_field to 1
# print the bit_field

In [57]:
from helper import hash256, hash160
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    for hash_function in (hash256, hash160):
        h = hash_function(item)
        bit = int.from_bytes(h, 'big') % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [58]:
from helper import murmur3
from bloomfilter import BIP37_CONSTANT
field_size = 2
num_functions = 2
tweak = 42
bit_field_size = field_size * 8
bit_field = [0] * bit_field_size
for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 2

Given a Bloom Filter with size=10, function count=5, tweak=99, what are the bytes that are set after adding these items? (use `bit_field_to_bytes` to convert to bytes)

* `b'Hello World'`
* `b'Goodbye!'`

In [59]:
# Exercise 2

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')
# calculate the bitfield size
# create an empty bit field
# loop through items
    # loop through function count
        # calculate the seed
        # get the murmur3 hash of the item using the seed
        # mod by the bitfield size
        # set the bit
# convert the bit field to bytes
# print the bytes in hex

### Exercise 3

Write the `add` method for `BloomFilter`

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_add`

In [60]:
# Exercise 3

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_add"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 4

Write the  `filterload` payload from the `BloomFilter` class.

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_filterload`

In [61]:
# Exercise 4

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_filterload"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 5

Write the `serialize` method for the `GetDataMessage` class.

#### Make [this test](/edit/code-ch12/network.py) pass: `network.py:GetDataMessageTest:test_serialize`

In [62]:
# Exercise 5

reload(network)
run(network.GetDataMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [63]:
from bloomfilter import BloomFilter
from helper import decode_base58
from merkleblock import MerkleBlock
from network import FILTERED_BLOCK_DATA_TYPE, GetHeadersMessage, GetDataMessage, HeadersMessage, SimpleNode
from tx import Tx

last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(address)

print('connecting to testnet node...')
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True, logging=False)

print('creating bloom filter and adding address...')
bf = BloomFilter(size=30, function_count=5, tweak=90210)
bf.add(h160)

print('handshaking...')
node.handshake()
print('loading bloom filter (filterload)...')
node.send(bf.filterload())

print('requesting headers from', last_block_hex)
start_block = bytes.fromhex(last_block_hex)
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)
headers = node.wait_for(HeadersMessage)
print('received', len(headers.blocks), 'headers')

getdata = GetDataMessage()
for idx, b in enumerate(headers.blocks):
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
print('built getdata for', len(getdata.data), 'filtered blocks, sending...')
node.send(getdata)

found = False
while not found:
    print('waiting for merkleblock or tx...')
    message = node.wait_for(MerkleBlock, Tx)
    print('got message:', message.command)
    if message.command == b'merkleblock':
        print('verifying merkleblock...')
        if not message.is_valid():
            raise RuntimeError('invalid merkle proof')
        print('merkleblock is valid')
    else:
        print('scanning tx outputs for address', address)
        for i, tx_out in enumerate(message.tx_outs):
            if tx_out.script_pubkey.address(testnet=True) == address:
                print('found: {}:{}'.format(message.id(), i))
                found = True
                break

connecting to testnet node...
creating bloom filter and adding address...
handshaking...
loading bloom filter (filterload)...
requesting headers from 00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19
received 2000 headers
built getdata for 2000 filtered blocks, sending...
waiting for merkleblock or tx...
got message: b'merkleblock'
verifying merkleblock...
merkleblock is valid
waiting for merkleblock or tx...
got message: b'tx'
scanning tx outputs for address mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv
found: e3930e1e566ca9b75d53b0eb9acb7607f547e1182d1d22bd4b661cfe18dcddf1:0


### Exercise 6

Get the current testnet block ID, send yourself some testnet coins, find the UTXO corresponding to the testnet coin _without using a block explorer_, create a transaction using that UTXO as an input and broadcast the `tx` message on the  testnet network.

In [64]:
# Exercise 6

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import (
    decode_base58,
    encode_varint,
    hash256,
    little_endian_to_int,
    read_varint,
)
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut

# answers.py の値を使用（本の例と同じ）
last_block_hex = '00000000000000a03f9432ac63813c6710bfe41712ac5ef6faab093fe2917636'
secret = little_endian_to_int(hash256(b'Jimmy Song'))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
h160 = decode_base58(addr)

print('using address:', addr)

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# connect to testnet.programmingbitcoin.com in testnet mode
print('connecting to testnet node...')
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True, logging=False)

# create a bloom filter of size 30 and 5 functions. Add a tweak.
print('creating bloom filter and adding address...')
bf = BloomFilter(30, 5, 90210)
# add the h160 to the bloom filter
bf.add(h160)

# complete the handshake
print('handshaking...')
node.handshake()
# load the bloom filter with the filterload command
print('loading bloom filter (filterload)...')
node.send(bf.filterload())

# set start block to last_block from above
start_block = bytes.fromhex(last_block_hex)
# send a getheaders message with the starting block
print('requesting headers from', last_block_hex)
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)

# wait for the headers message
headers = node.wait_for(HeadersMessage)
print('received', len(headers.blocks), 'headers')

# store the last block as None
last_block = None
# initialize the GetDataMessage
getdata = GetDataMessage()
# loop through the blocks in the headers
for b in headers.blocks:
    # check that the proof of work on the block is valid
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    # check that this block's prev_block is the last block
    if last_block is not None and b.prev_block != last_block:
        raise RuntimeError('chain broken')
    # add a new item to the get_data_message
    # should be FILTERED_BLOCK_DATA_TYPE and block hash
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
    # set the last block to the current hash
    last_block = b.hash()

print('built getdata for', len(getdata.data), 'filtered blocks, sending...')
# send the getdata message
node.send(getdata)

# initialize prev_tx, prev_index, prev_amount to None
prev_tx, prev_index, prev_amount = None, None, None
# loop while prev_tx is None 
while prev_tx is None:
    # wait for the merkleblock or tx commands
    print('waiting for merkleblock or tx...')
    message = node.wait_for(MerkleBlock, Tx)
    print('got message:', message.command)
    # if we have the merkleblock command
    if message.command == b'merkleblock':
        # check that the MerkleBlock is valid
        print('verifying merkleblock...')
        if not message.is_valid():
            raise RuntimeError('invalid merkle proof')
        print('merkleblock is valid')
    # else we have the tx command
    else:
        # set the tx's testnet to be True
        message.testnet = True
        # loop through the tx outs
        print('scanning tx outputs for address', addr)
        for i, tx_out in enumerate(message.tx_outs):
            # if our output has the same address as our address we found it
            if tx_out.script_pubkey.address(testnet=True) == addr:
                # we found our utxo. set prev_tx, prev_index, and amount
                prev_tx = message.hash()
                prev_index = i
                prev_amount = tx_out.amount
                print('found utxo:', prev_tx.hex(), prev_index, 'amount:', prev_amount)
                break

# create the TxIn
tx_in = TxIn(prev_tx, prev_index)
# calculate the output amount (previous amount minus the fee)
output_amount = prev_amount - fee
print('output amount:', output_amount)
# create a new TxOut to the target script with the output amount
tx_out = TxOut(output_amount, target_script)
# create a new transaction with the one input and one output
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the only input of the transaction
print('signing input...')
print('sign result:', tx_obj.sign_input(0, private_key))
# serialize and hex to see what it looks like
raw_tx = tx_obj.serialize().hex()
print('raw tx:', raw_tx)
# send this signed transaction on the network
print('broadcasting tx...')
node.send(tx_obj)
# wait a sec so this message goes through with time.sleep(1) 
time.sleep(1)
# now ask for this transaction from the other node
print('requesting tx back to confirm broadcast...')
getdata = GetDataMessage()
# ask for our transaction by adding it to the message
getdata.add_data(TX_DATA_TYPE, tx_obj.hash())
# send the message
node.send(getdata)
# now wait for a Tx response
received_tx = node.wait_for(Tx)
# if the received tx has the same id as our tx, we are done!
if received_tx.id() == tx_obj.id():
    print('success!')

using address: mseRGXB89UTFVkWJhTRTzzZ9Ujj4ZPbGK5
connecting to testnet node...
creating bloom filter and adding address...
handshaking...
loading bloom filter (filterload)...
requesting headers from 00000000000000a03f9432ac63813c6710bfe41712ac5ef6faab093fe2917636
received 2000 headers
built getdata for 2000 filtered blocks, sending...
waiting for merkleblock or tx...
got message: b'merkleblock'
verifying merkleblock...
merkleblock is valid
waiting for merkleblock or tx...
got message: b'tx'
scanning tx outputs for address mseRGXB89UTFVkWJhTRTzzZ9Ujj4ZPbGK5
found utxo: b2cddd41d18d00910f88c31aa58c6816a190b8fc30fe7c665e1cd2ec60efdf3f 7 amount: 15026203
output amount: 15021203
signing input...
sign result: True
raw tx: 01000000013fdfef60ecd21c5e667cfe30fcb890a116688ca51ac3880f91008dd141ddcdb2070000006b483045022100ff77d2559261df5490ed00d231099c4b8ea867e6ccfe8e3e6d077313ed4f1428022033a1db8d69eb0dc376f89684d1ed1be75719888090388a16f1e8eedeb8067768012103dc585d46cfca73f3a75ba1ef0c5756a21c19245

In [65]:
# Compute TxID for the raw transaction from Exercise 6
from helper import hash256

raw_hex = "01000000013fdfef60ecd21c5e667cfe30fcb890a116688ca51ac3880f91008dd141ddcdb2070000006b483045022100ff77d2559261df5490ed00d231099c4b8ea867e6ccfe8e3e6d077313ed4f1428022033a1db8d69eb0dc376f89684d1ed1be75719888090388a16f1e8eedeb8067768012103dc585d46cfca73f3a75ba1ef0c5756a21c1924587480700c6eb64e3f75d22083ffffffff019334e500000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000"

raw_bytes = bytes.fromhex(raw_hex)
txid = hash256(raw_bytes)[::-1].hex()
print("TxID:", txid)

TxID: 380c9484d577eec7339c98b5c09d7ab72561ff9344ac817c3aef7a9e05d48663
